# TASK

**На основе T5 (или любой другой) модели из библиотеки HuggingFace сделать переводчик**

# INIT BLOCK

### INSTALL

In [ ]:
!pip install transformers

### IMPORT REQUIREMENTS

In [ ]:
from transformers import (pipeline, Pipeline, AutoConfig, AutoTokenizer, AutoProcessor, AutoModelForSeq2SeqLM, TranslationPipeline)

# SRC

## CONSTANTS

In [ ]:
T5_LARGE_MODEL: str = "t5-large"
T5_SMALL_MODEL: str = "t5-small"
# FACEBOOK_NLLB: str = "facebook/nllb-200-3.3B"  # Не висточає RAM
# FACEBOOK_NLLB: str = "facebook/nllb-200-distilled-1.3B"  # Не висточає RAM
FACEBOOK_NLLB: str = "facebook/nllb-200-distilled-600M"

DEVICE: str = "cpu"
MODEL_SAVE_PATH: str = "./models/"

ENG: str = "en"
UKR: str = "uk"

## TRANSLATOR

In [ ]:
class Translator:

    def __init__(
            self,
            model=T5_SMALL_MODEL,
            task="translation_en_to_de",
            tokenizer=AutoTokenizer,
            model_config=AutoConfig,
            processor=AutoProcessor,
    ):
        model_config = self.configurate_recognizer_part(recognizer_part_class=model_config, model=model)
        tokenizer = self.configurate_recognizer_part(recognizer_part_class=tokenizer, model=model)
        processor = self.configurate_recognizer_part(recognizer_part_class=AutoProcessor, model=model)
        model = self.configurate_recognizer_part(recognizer_part_class=AutoModelForSeq2SeqLM, model=model)

        self.translation_pipeline: Pipeline = pipeline(
            task=task,
            config=model_config,
            tokenizer=tokenizer,
            model=model,
            device=DEVICE,
        )

    @staticmethod
    def configurate_recognizer_part(recognizer_part_class, model):
        local_model_path: str = f"{MODEL_SAVE_PATH}{model}"

        try:
            return recognizer_part_class.from_pretrained(pretrained_model_name_or_path=local_model_path)

        except Exception as exception:
            print(type(exception), exception)
            part = recognizer_part_class.from_pretrained(model)
            part.save_pretrained(save_directory=local_model_path)
            return recognizer_part_class.from_pretrained(pretrained_model_name_or_path=local_model_path)

    def translate(self, text: str, **kwargs):
        print(self.translation_pipeline(text, **kwargs))

    def translate_sequence(self, text: list, **kwargs):
        for sentence in text:
            print(self.translation_pipeline(sentence, **kwargs))

# TRANSLATE

## T5 SMALL

### INIT MODEL

In [ ]:
t5_small_model: Translator = Translator()

### TRANSLATE

In [ ]:
t5_small_model.translate(text="Hello! Good Morning! My name is Daniil Omelianenko! I'm Ukrainian!")
t5_small_model.translate(text="Hello!")
t5_small_model.translate(text="Good Morning!")
t5_small_model.translate(text="My name is Daniil Omelianenko!")
t5_small_model.translate(text="I'm Ukrainian!")

[{'translation_text': 'Ich bin ukrainisch!'}]
[{'translation_text': 'Hallo!'}]
[{'translation_text': 'Guter Morgen!'}]
[{'translation_text': 'Mein Name ist Daniil Omelianenko!'}]
[{'translation_text': 'Ich bin Ukrainer!'}]


t5 small не може перекласти декілька реченнь одночасно тому переводимо по частинам:

In [ ]:
t5_small_model.translate_sequence(text="Hello! Good Morning! My name is Daniil Omelianenko! I'm Ukrainian!".split(sep=" "))

[{'translation_text': 'Hallo!'}]
[{'translation_text': 'Gutes'}]
[{'translation_text': 'Morgen!'}]
[{'translation_text': 'Mein'}]
[{'translation_text': 'Namen'}]
[{'translation_text': 'ist'}]
[{'translation_text': 'Daniil'}]
[{'translation_text': 'Omelianenko!'}]
[{'translation_text': 'Ich bin'}]
[{'translation_text': 'Ukrainer!'}]


## T5 LARGE

### INIT MODEL

In [ ]:
t5_large_model: Translator = Translator(model=T5_LARGE_MODEL)

<class 'OSError'> Can't load the configuration of './models/t5-large'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './models/t5-large' is the correct path to a directory containing a config.json file


<class 'OSError'> Can't load tokenizer for './models/t5-large'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './models/t5-large' is the correct path to a directory containing all relevant files for a T5TokenizerFast tokenizer.


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


<class 'OSError'> Error no file named pytorch_model.bin, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory ./models/t5-large.


### TRANSLATE

In [ ]:
t5_large_model.translate(text="Hello! Good Morning! My name is Daniil Omelianenko! I'm Ukrainian!")

Hallo, guten Morgen, mein Name ist Daniil Omelianenko, ich bin Ukrainin!


In [ ]:
anthem: str = """Nay, thou art not dead, Ukraine,
See, the glory’s born again,
And the skies, O brethren,
Smile once more!"""
t5_large_model.translate(text=anthem)

Nein, du bist nicht tot, Ukraine, Seht, die Herrlichkeit ist wiedergeboren, Und der Himmel, o Brüder, lächelt noch einmal!


## FACEBOOK_NLLB

### INIT MODEL

In [ ]:
facebook_nllb_model: Translator = Translator(model=FACEBOOK_NLLB, task="translation")

### TRANSLATE

In [ ]:
anthem_ukr: str = """
Ще не вмерла України і слава, і воля.
Ще нам, браття молодії, усміхнеться доля.
Згинуть наші вороженьки, як роса на сонці,
Запануєм і ми, браття, у своїй сторонці.

Приспів:
Душу й тіло ми положим за нашу свободу,
І покажем, що ми, браття, козацького роду.

Станем, браття, в бій кривавий від Сяну до Дону,
В ріднім краю панувати не дамо нікому;
Чорне море ще всміхнеться, дід Дніпро зрадіє,
Ще у нашій Україні доленька наспіє.

Приспів.
А завзяття, праця щира свого ще докаже,
Ще ся волі в Україні піснь гучна розляже,
За Карпати відоб’ється, згомонить степами,
України слава стане поміж ворогами.

Приспів.
"""

# facebook_nllb_model.translate(anthem_ukr, src_lang=UKR, tgt_lang=ENG, max_length=400)
facebook_nllb_model.translation_pipeline(anthem_ukr, src_lang=ENG, tgt_lang=UKR, max_length=400)

[{'translation_text': "ukranina,success,and will will never die.we will, young brothers, smile for fate.we will die our bridesmaids, like a rose in the sun,zapanuim and we, brothers, on its side.song: we will lay our soul and body for our freedom,and show that we, brethren, of the Cossack family.let us, brethren, in a war from the crooked to the Don, in the southern edge, dominate no one;the black sea will smile, grandfather Dnipro betrays,we will have our Ukraine's share of the crown.song.apazyatya, stretch my shield, will sit in Ukraine will sing a song of glory,the carpathians will be welcome,they will be the enemy of Ukraine."}]

# Висновок

Зробив 3 перекладача з різних моделей:
1. T5 SMALL: перекладає тільки короткі частини реченнь, або по одному реченню за раз. Працює тільки з кількома мовами (English, French, Romanian, German).
2. T5 LARGE: може перекладати великі речення, та тексти. Працює тільки з кількома мовами (English, French, Romanian, German).
3. FACEBOOK_NLLB: Перекладає будь які тексти. Може перекладати на 196 мов, включаючи Українську. Недолік лише один - потребує дуже багато ресурсів.